# 분류
- 범주 예측
  + 이진 분류 : 남/여 분류 , 대출 연체 여부
  + 다중 분류 : 등급 분류, 이밈지와 품종 분류

In [1]:
# 붓꽃 분류
from sklearn import datasets
import numpy as np

iris = datasets.load_iris()
x = iris.data[:, [2,3]]
y = iris.target

x.shape, y.shape

((150, 2), (150,))

In [2]:
# 0 = setosa, 1 = versicolor, 2 = virginica
# 다중 분류
print(np.unique(y))

[0 1 2]


In [3]:
from sklearn.model_selection import train_test_split

# x = 특성 feature 데이터
# y = 레이블 label 데이터
# target
x_train, x_test, y_train, y_test = train_test_split(
    x,y, test_size = 0.3, random_state = 1            # random_stat 숫자는 무의미, 아무 숫자나 가능
    ,stratify = y                                     # 층하추출
)

x_train.shape, x_test.shape, y_train.shape, y_test.shape


((105, 2), (45, 2), (105,), (45,))

- 변수 차이
  + 26일 코드 x_val
    + 모델 한번에 결정 안함 (검증 필요, 실험을 하겠다.)
    + 테스트 데이터가 확보된 상태

  + 27일 (위에) 코드 x_test
     + 테스트 데이터가 없음. 임의로 만듬
     + 모델 검증 안함 (바로 사용하겠다.)

- 특성 표준화

In [4]:
# 모든 수치 데이터를 표준화 평균 0, 표준편차 1로 만들겠음
# 단위가 서로 다를 때 한다.
# linear model 활용 시 필수 (linear model == 수식이 들어간 모델)
# 결정트리와 비선형모델 은 (특성표준화를 할)필요가 없다.
from sklearn.preprocessing import StandardScaler

sc = StandardScaler() # 인스턴스 (객체) 생성
sc.fit(x_train)

# 훈련 데이터와 테스트 데이터를 표준화
# 테스트 데이터는 미지의 데이터 (모수를 모르는 데이터)
# 연구자가 아는 건 훈련 데이터만 파악된 상태
x_train_std = sc.transform(x_train)
x_test_std = sc.transform(x_test)

In [5]:
# from sklearn.preprocessing import StandardScaler

# sc = StandardScaler()
# sc.fit(x_trian)
# # sc.fit(x_test) # 해당 코드를 입력하면 미래의 데이터를 알고있다는 뜻이 됨.
# x_train_std = sc.transform(x_train)

# (위 코드 결과)
# 과대적함 발생 (너무 잘 맞춘다는 뜻)
# data leakage 상태다라고 말할 수 있음.

In [6]:
# 모델 만들기

from sklearn.linear_model import Perceptron

# eta0 = 0.1 학습률 설정
# 하이퍼파라미터 튜닝

# 퍼셉트론 : 딥러닝 모델의 초창기 모델 (1950년대)
ppn = Perceptron(eta0=0.1, random_state = 1)
ppn.fit(x_train_std, y_train)

Perceptron(eta0=0.1, random_state=1)

In [7]:
# 테스트 데이터에 대한 예측 수행

y_pred = ppn.predict(x_test_std)
y_pred

array([2, 0, 0, 2, 1, 1, 2, 1, 2, 0, 0, 2, 0, 1, 0, 1, 2, 1, 1, 2, 2, 0,
       1, 2, 1, 1, 0, 2, 0, 2, 0, 0, 1, 1, 2, 2, 0, 0, 0, 1, 2, 2, 1, 0,
       0])

In [8]:
# 실제 레이블과 예측값이 다른 샘플의 개수 출력
# 1개가 다름 (결과값)
(y_test != y_pred).sum()

np.int64(1)

In [9]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9777777777777777

In [10]:
import seaborn as sns

# 분류모델 만들기
tips = sns.load_dataset("tips")
tips.head()

# size 범주를 예측하는 모델 만들기
# (가이드) 문자열 데이터 많음 -> 인코딩 변환해야함. ,수치데이터 -> 표준화(스케일링), 훈련데이터/테스트데이터 분리해야함
tips['size'].unique() # 다중분류
tips['size'].value_counts()

size
2    156
3     38
4     37
5      5
1      4
6      4
Name: count, dtype: int64

In [13]:
import pandas as pd
from sklearn.metrics import accuracy_score 
import seaborn as sns 
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd 
import matplotlib.pyplot as plt

categorical_features = ['sex', 'smoker', 'day', 'time']
numeric_features = ['total_bill', 'tip']

X = tips[numeric_features + categorical_features]
y = tips['size']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 수치형 데이터 전처리
scaler = StandardScaler()
X_train_numeric = scaler.fit_transform(X_train[numeric_features])
X_test_numeric = scaler.transform(X_test[numeric_features])

# X_dummies = pd.get_dummies(X[categorical_features])
X_train_categoricals = pd.get_dummies(X_train[categorical_features])
X_test_categoricals = pd.get_dummies(X_test[categorical_features])

X_train_processed = np.hstack([X_train_numeric, X_train_categoricals])
X_test_processed = np.hstack([X_test_numeric, X_test_categoricals])

ppn = Perceptron(eta0=0.1, random_state=1) 
ppn.fit(X_train_processed, y_train)

# 테스트 데이터에 대한 예측 수행 
# 에러 발생
y_pred = ppn.predict(X_test_processed)
print((y_test != y_pred).sum())
print(accuracy_score(y_test, y_pred))

# 여기 적용이 안됨
new_data = pd.DataFrame({
    'total_bill': [20.0],
    'tip': [4.0],
    'sex': ['Male'],
    'smoker': ['No'],
    'day': ['Sun'],
    'time': ['Dinner']
})

# 수치형 데이터 전처리
new_numeric = scaler.transform(new_data[numeric_features])
new_categorical = pd.get_dummies(new_data[categorical_features])
new_processed = np.hstack([new_numeric, new_categorical])
ppn.predict(new_processed)


## 훈련데이터와 테스트데이터로 분리 후 처리

16
0.673469387755102


ValueError: X has 6 features, but Perceptron is expecting 12 features as input.

In [ ]:
/*
    iris data classification sklearn, data leakage, pipeline
    cross validation, hyper parameter tuning 추가   
*/